In [675]:
import pandas as pd
import numpy as np

In [676]:
parts_data = pd.read_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/final_project/data/Project_Data/parts_analysis_post.csv')
time_data_post = pd.read_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/final_project/data/Project_Data/time_data_post.csv')
time_data_pre = pd.read_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/final_project/data/Project_Data/time_data_pre.csv')
pre_merge_data = pd.read_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/final_project/data/Project_Data/parts_analysis_pre.csv')
purchased_data = pd.read_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/final_project/data/Project_Data/purchased_parts_ytd.csv', dtype='object')


In [677]:
time_data_pre.drop(columns=['Description'], inplace=True)
purchased_data.drop(columns=['Description'], inplace=True)

In [678]:
def preprocess_dataframe(df):
    """
        CSV files downloaded from ERP had empty columns and rows so this just drops them
    """
    df = df.dropna(axis=1, how='all')
    df = df.dropna(how='all')    
    return df

parts_data = preprocess_dataframe(parts_data)
time_data_pre = preprocess_dataframe(time_data_pre)
time_data_post = preprocess_dataframe(time_data_post)
pre_merge_data = preprocess_dataframe(pre_merge_data)
purchased_data = preprocess_dataframe(purchased_data)

In [679]:
purchased_data[['Qnty Ordered', 'Qnty Recieved']] = np.where(purchased_data[['Qnty Ordered', 'Qnty Recieved']].isna(), 0, purchased_data[['Qnty Ordered', 'Qnty Recieved']])
purchased_data.rename(columns={
    'Qnty Ordered': 'Quantity Ordered YTD', 
    'Qnty Recieved': 'Quantity Received YTD',
    'Part #': 'Part Number'
}, inplace=True)

In [680]:
purchased_data[['Quantity Ordered YTD', 'Quantity Received YTD']].value_counts()

Quantity Ordered YTD  Quantity Received YTD
1                     1                        1915
0                     0                        1085
Qnty Ordered          Qnty Recieved             540
2                     2                         421
1                     0                         229
                                               ... 
                      29.9                        1
                      29.52                       1
                      29.22                       1
                      288.97                      1
                      65.09                       1
Length: 1651, dtype: int64

In [681]:
qty_ordered_rows_mask = purchased_data['Quantity Ordered YTD'] == 'Qnty Ordered' 
qty_recieved_rows_mask = (purchased_data['Quantity Received YTD'] == 'Qnty Recieved') | (purchased_data['Quantity Received YTD'] == 'Extended Cost')

# Drop the rows using the boolean mask
purchased_data.drop(purchased_data[qty_ordered_rows_mask].index, inplace=True)
purchased_data.drop(purchased_data[qty_recieved_rows_mask].index, inplace=True)

# Print the resulting DataFrame
purchased_data[['Quantity Ordered YTD', 'Quantity Received YTD']].value_counts()


/var/folders/59/vc05kjdn7s14r9w654dq4xlh0000gn/T/ipykernel_84114/1289788845.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  purchased_data.drop(purchased_data[qty_recieved_rows_mask].index, inplace=True)


Quantity Ordered YTD  Quantity Received YTD
1                     1                        1915
0                     0                        1085
2                     2                         421
1                     0                         229
3                     3                         152
                                               ... 
1                     3.36                        1
                      3.29                        1
                      3.2                         1
                      3.04                        1
                      65.09                       1
Length: 1648, dtype: int64

In [682]:
purchased_data.dropna(inplace=True)

In [683]:
purchased_data.isna().value_counts()

Part Number  Quantity Ordered YTD  Quantity Received YTD
False        False                 False                    2917
dtype: int64

In [684]:
print(f'Pre Data: {time_data_pre.shape}')
print(f'Post Data: {time_data_post.shape}')
print(f'Parts Data: {parts_data.shape}')
print(f'Pre Merge Data: {pre_merge_data.shape}')
print(f'Purchased Data: {purchased_data.shape}')



Pre Data: (68966, 19)
Post Data: (36066, 19)
Parts Data: (69020, 20)
Pre Merge Data: (36066, 20)
Purchased Data: (2917, 3)


In [685]:
purchased_data.columns

Index(['Part Number', 'Quantity Ordered YTD', 'Quantity Received YTD'], dtype='object')

In [686]:
index_to_drop_ = 68965
parts_data.drop(index_to_drop_, axis=0, inplace=True)
index_to_drop = 36065
pre_merge_data.drop(index_to_drop, axis=0, inplace=True)


In [687]:
parts_data.dtypes

Part Number          object
Category             object
Description          object
Supplier Name        object
Margin %             object
Margin               object
Active Price         object
On Hand             float64
Qty Sold Last       float64
Qty Sold Last .1    float64
Qty Sold Last .2    float64
Qty Sold Last .3    float64
Qty Sold Last .4    float64
Qty Sold Last .5    float64
Qty Sold Last .6    float64
Qty Sold Last .7    float64
Qty Sold Last .8    float64
Qty Sold Last .9    float64
Months No           float64
Reorder Point       float64
dtype: object

In [688]:
import pandas as pd

def clean_and_convert_columns(dataframes, columns, target_dtype):
    """
    Clean and convert specified columns in multiple dataframes to the target data type.

    Args:
        dataframes (list of pd.DataFrame): List of dataframes to clean and convert columns.
        columns (list of str): List of column names to clean and convert.
        target_dtype (type): Target data type to convert columns to.

    Returns:
        list of pd.DataFrame: List of cleaned dataframes with specified columns converted.
    """
    cleaned_dataframes = []

    for df in dataframes:
        for column in columns:
            if column in df.columns:
                df[column] = df[column].str.replace(',', '')
                if column.endswith('%'):
                    df[column] = df[column].str.rstrip('%')
                df[column] = df[column].astype(target_dtype)
        
        cleaned_dataframes.append(df)

    return cleaned_dataframes

dataframes = [pre_merge_data, parts_data]

# Columns to clean and convert
columns_to_clean = ['Active Price', 'Margin', 'Margin %']

# Clean and convert columns to float
cleaned_dataframes = clean_and_convert_columns(dataframes, columns_to_clean, float)

In [689]:
#merge the pre_merge_data with parts_data and additively fix the data
"""
Necessary because the store went through a mid-year merger and could not aggregate all sales data
"""

parts_data_grouped = pd.concat([parts_data, pre_merge_data])

time_data_grouped = pd.concat([time_data_pre, time_data_post])

#add pre and post sales data
grouped_columns = ['Part Number', 'Category', 'Description', 
                   'Supplier Name', 'Margin %',
                    'Margin', 'Active Price']

parts_data = parts_data_grouped.groupby(grouped_columns, as_index=False).sum()
#add pre and post time data
time_data = time_data_grouped.groupby('Part Number', as_index=False).sum()


In [690]:
#ensure addition worked

# Calculate the sum of the 'Margin' column in the original parts_data and time_data
original_parts_data_margin_sum = parts_data_grouped['Margin'].sum()

# Calculate the sum of the 'Margin' column in the merged and grouped parts_data
merged_parts_data_margin_sum = parts_data['Margin'].sum()

print(f'Original: {original_parts_data_margin_sum}')
print(f'After Sum: {merged_parts_data_margin_sum}')

# Compare the sums
if original_parts_data_margin_sum == merged_parts_data_margin_sum:
    print("Margin sums match after operations.")
else:
    print("Margin sums do not match after operations.")



Original: 53665421.068300016
After Sum: 53663209.9183
Margin sums do not match after operations.


In [691]:
#merge all the data into 1 set
parts_data = pd.merge(parts_data, time_data, on='Part Number')

In [692]:
#merge ordered data:
parts_data = pd.merge(parts_data, purchased_data, on='Part Number', how='left')

parts_data[['Quantity Ordered YTD', 'Quantity Received YTD']] = np.where(parts_data[['Quantity Ordered YTD', 'Quantity Received YTD']].isna(), 0, parts_data[['Quantity Ordered YTD', 'Quantity Received YTD']])

parts_data[['Quantity Ordered YTD', 'Quantity Received YTD']] = round(parts_data[['Quantity Ordered YTD', 'Quantity Received YTD']].astype(float),0)
parts_data[['Quantity Ordered YTD', 'Quantity Received YTD']] = parts_data[['Quantity Ordered YTD', 'Quantity Received YTD']].astype(int)

In [693]:
parts_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103359 entries, 0 to 103358
Data columns (total 40 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Part Number            103359 non-null  object 
 1   Category               103359 non-null  object 
 2   Description            103359 non-null  object 
 3   Supplier Name          103359 non-null  object 
 4   Margin %               103359 non-null  float64
 5   Margin                 103359 non-null  float64
 6   Active Price           103359 non-null  float64
 7   On Hand                103359 non-null  float64
 8   Qty Sold Last          103359 non-null  float64
 9   Qty Sold Last .1       103359 non-null  float64
 10  Qty Sold Last .2       103359 non-null  float64
 11  Qty Sold Last .3       103359 non-null  float64
 12  Qty Sold Last .4       103359 non-null  float64
 13  Qty Sold Last .5       103359 non-null  float64
 14  Qty Sold Last .6       103359 non-nu

In [694]:
parts_data.rename(columns={
    'Active Price': 'Price',
    'On Hand ': 'Quantity',
    'Qty Sold Last ': 'Sales Last Month', 
    'Qty Sold Last .1': 'Sales Last 3 Months', 
    'Qty Sold Last .2': 'Sales Last 6 Months', 
    'Qty Sold Last .3': 'Sales Last 9 Months', 
    'Qty Sold Last .4': 'Sales Last 12 Months', 
    'Qty Sold Last .5': 'Sales Last 2 Years', 
    'Qty Sold Last .6': 'Sales Last 3 Years', 
    'Qty Sold Last .7': 'Sales Last 4 Years', 
    'Qty Sold Last .8': 'Sales Last 5 Years',
    'Qty Sold Last .9': 'Sales Last 10 Years',
    'Months No ': 'Months No Sale'
}, inplace=True)


In [695]:
cols_to_rename = ['Ja', 'Fe','Ma', 'Ap',  'My',
       'Ju', 'Jl', 'Au', 'Se', 'Oc', 'No', 'De',
       '1st Qtr', '2nd Qtr', '3rd Qtr', '4th Qtr',
       'This Year', 'Last Year']

# Define a mapping of current column names to new column names
column_mapping = {col: f'Sales - {col}'for col in cols_to_rename}

# Rename the columns using the .rename() method
parts_data.rename(columns=column_mapping, inplace=True)

parts_data.rename(columns={
    'Sales - Ja': 'Sales - Jan', 
    'Sales - Fe': 'Sales - Feb', 
    'Sales - Ma': 'Sales - Mar', 
    'Sales - Ap': 'Sales - Apr', 
    'Sales - My': 'Sales - May', 
    'Sales - Ju': 'Sales - Jun', 
    'Sales - Jl': 'Sales - Jul',
    'Sales - Au': 'Sales - Aug', 
    'Sales - Se': 'Sales - Sep', 
    'Sales - Oc': 'Sales - Oct', 
    'Sales - No': 'Sales - Nov', 
    'Sales - De': 'Sales - Dec',
}, inplace=True)

parts_data.columns


Index(['Part Number', 'Category', 'Description', 'Supplier Name', 'Margin %',
       'Margin', 'Price', 'Quantity', 'Sales Last Month',
       'Sales Last 3 Months', 'Sales Last 6 Months', 'Sales Last 9 Months',
       'Sales Last 12 Months', 'Sales Last 2 Years', 'Sales Last 3 Years',
       'Sales Last 4 Years', 'Sales Last 5 Years', 'Sales Last 10 Years',
       'Months No Sale', 'Reorder Point', 'Sales - Jan', 'Sales - Feb',
       'Sales - Mar', 'Sales - Apr', 'Sales - May', 'Sales - Jun',
       'Sales - Jul', 'Sales - Aug', 'Sales - Sep', 'Sales - Oct',
       'Sales - Nov', 'Sales - Dec', 'Sales - 1st Qtr', 'Sales - 2nd Qtr',
       'Sales - 3rd Qtr', 'Sales - 4th Qtr', 'Sales - This Year',
       'Sales - Last Year', 'Quantity Ordered YTD', 'Quantity Received YTD'],
      dtype='object')

In [696]:
parts_data[['Part Number', 'Description','Price', 'Margin', 'Quantity','Sales Last Month', 'Sales Last 3 Months',
       'Sales Last 6 Months', 'Sales Last 9 Months','Sales Last 12 Months']]

,Part Number,Description,Price,Margin,Quantity,Sales Last Month,Sales Last 3 Months,Sales Last 6 Months,Sales Last 9 Months,Sales Last 12 Months
0,0,RETURN SPRING,7.26,3.77,0.0,0.0,0.0,0.0,0.0,0.0
1,0 580 463 999,EURO MOTORELECTRICS BOSCH,629.99,397.99,0.0,0.0,0.0,0.0,0.0,0.0
2,0........0,0........0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
3,00-0450,"SCREW, GEARBOX,INSPEC&",1.95,1.49,0.0,0.0,0.0,0.0,0.0,0.0
4,00-548588,Tube 150/90- 15 centre90' stem,19.99,5.60,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
103354,t2048903,"Yoke Assy, Lower",392.99,125.30,0.0,0.0,0.0,0.0,0.0,0.0
103355,t2501733,YTX12-BS,100.99,51.00,0.0,0.0,0.0,0.0,1.0,1.0
103356,t3010108,Flanged Sleeve M6,2.89,0.90,0.0,0.0,0.0,0.0,0.0,0.0
103357,t3330696,screw,5.99,2.83,0.0,0.0,0.0,0.0,0.0,0.0


In [697]:
parts_data['Cost Per Unit'] = parts_data['Price'] - parts_data['Margin']
parts_data['Total Cost'] = parts_data['Cost Per Unit'] * abs(parts_data['Quantity'])

In [698]:
def fix_text(data_frame, column_name):
    if (column_name != 'Part Number') & (column_name != 'Category'):
        data_frame[column_name] = data_frame[column_name].str.lower().str.strip()
    else:
        data_frame[column_name] = data_frame[column_name].str.strip()
    
    return data_frame

text_cols = parts_data.select_dtypes(exclude='number').columns
for col in text_cols:
    parts_data = fix_text(parts_data, col)

In [699]:
parts_data.head()

,Part Number,Category,Description,Supplier Name,Margin %,Margin,Price,Quantity,Sales Last Month,Sales Last 3 Months,...,Sales - 1st Qtr,Sales - 2nd Qtr,Sales - 3rd Qtr,Sales - 4th Qtr,Sales - This Year,Sales - Last Year,Quantity Ordered YTD,Quantity Received YTD,Cost Per Unit,Total Cost
0,0,PM1,return spring,triumph motorcycles,51.93,3.77,7.26,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,3.49,0.0
1,0 580 463 999,ACC,euro motorelectrics bosch,euro motorelectrics,63.17,397.99,629.99,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,232.00,0.0
2,0........0,ACC,0........0,parts canada/power twins,0.00,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.00,0.0
3,00-0450,ACC,"screw, gearbox,inspec&",motovan,76.41,1.49,1.95,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.46,0.0
4,00-548588,ATA,tube 150/90- 15 centre90' stem,butler maps inc,28.01,5.60,19.99,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,14.39,0.0


In [700]:
parts_data.dropna(subset=['Part Number'], inplace=True)
parts_data.dropna(subset=['Price'], inplace=True)

In [701]:
def drop_negatives(data_frame, column_name, threshold=0):
    mask = data_frame[column_name] <= threshold
    if mask.any():
        data = data_frame[~mask]
        return data
    else:
        return data_frame

# Example usage
parts_data = drop_negatives(parts_data, 'Price')


In [702]:
parts_data['Obsolete'] = parts_data['Months No Sale'].apply(lambda x: 1 if x >= 12 else 0)

In [703]:
part_num = '2879572-156-P'
supplier = 'polaris acceptance'

parts_data.loc[parts_data['Part Number'] == part_num, 'Supplier Name'] = supplier
parts_data['Supplier Name'].value_counts()


bmw group canada                  23829
triumph motorcycles               14645
motovan                            9357
ducati                             9219
suzuki                             7021
                                  ...  
revzilla                              1
washington cycle works                1
roland sands designs (official        1
chesapeake cycles inc                 1
black widow (uk)                      1
Name: Supplier Name, Length: 280, dtype: int64

In [704]:
part_num = '0711 9978 396'
supplier = 'bmw group canada'

parts_data.loc[parts_data['Part Number'] == part_num, 'Supplier Name'] = supplier


In [705]:
#convert things to main suppliers and other suppliers just to simplify, then one hot encode everything
supplier_cost = parts_data.groupby('Supplier Name')['Total Cost'].sum().sort_values(ascending=False)
round(supplier_cost, 2)

Supplier Name
bmw group canada            527144.87
polaris acceptance          388657.78
triumph motorcycles         349799.67
ducati                      191993.31
parts canada/power twins    135997.98
                              ...    
gizmo mill                       0.00
got gear motorsports             0.00
gps city                         0.00
gt moto products                 0.00
ztechnik                         0.00
Name: Total Cost, Length: 280, dtype: float64

In [706]:
historical_cols = ['Sales Last Month',
       'Sales Last 3 Months', 'Sales Last 6 Months',
       'Sales Last 9 Months', 'Sales Last 12 Months', 
       'Sales Last 2 Years','Sales Last 3 Years', 
       'Sales Last 4 Years', 'Sales Last 5 Years', 'Sales Last 10 Years']

this_year_cols = ['Sales - Jan', 'Sales - Feb', 'Sales - Mar',
       'Sales - Apr', 'Sales - May', 'Sales - Jun', 'Sales - Jul',
       'Sales - Aug', 'Sales - Sep', 'Sales - Oct', 'Sales - Nov', 'Sales - Dec']


In [707]:
def calculate_metric(parts_data, time_frame_cols, metric, column_to_multiply, total_sales_year):
    for col in time_frame_cols:
        # Extract the time frame (e.g., 'Jan', 'Feb', etc.) from the column name
        time_frame = col.split('-')[-1].strip()

        # Create a new column name for the metric (e.g., 'Gross Profit - Jan', etc.)
        metric_col = f'{metric} - {time_frame}'

        # Calculate the metric for the current time frame
        parts_data[metric_col] = parts_data[column_to_multiply] * parts_data[col]

    parts_data[f'Total {metric}'] = parts_data[total_sales_year] * parts_data[column_to_multiply]

    return parts_data

# Calculate Revenue for historical time frames (assuming you have a column 'Cost' representing the cost of each part)
parts_data = calculate_metric(parts_data, historical_cols, 'Gross Profit', 'Price', 'Sales - Last Year')

# Calculate Profits for this year time frames (assuming you have a column 'Cost' representing the cost of each part)
parts_data = calculate_metric(parts_data, this_year_cols, 'Gross Profit', 'Price', 'Sales - This Year')

In [708]:
parts_data.columns

Index(['Part Number', 'Category', 'Description', 'Supplier Name', 'Margin %',
       'Margin', 'Price', 'Quantity', 'Sales Last Month',
       'Sales Last 3 Months', 'Sales Last 6 Months', 'Sales Last 9 Months',
       'Sales Last 12 Months', 'Sales Last 2 Years', 'Sales Last 3 Years',
       'Sales Last 4 Years', 'Sales Last 5 Years', 'Sales Last 10 Years',
       'Months No Sale', 'Reorder Point', 'Sales - Jan', 'Sales - Feb',
       'Sales - Mar', 'Sales - Apr', 'Sales - May', 'Sales - Jun',
       'Sales - Jul', 'Sales - Aug', 'Sales - Sep', 'Sales - Oct',
       'Sales - Nov', 'Sales - Dec', 'Sales - 1st Qtr', 'Sales - 2nd Qtr',
       'Sales - 3rd Qtr', 'Sales - 4th Qtr', 'Sales - This Year',
       'Sales - Last Year', 'Quantity Ordered YTD', 'Quantity Received YTD',
       'Cost Per Unit', 'Total Cost', 'Obsolete',
       'Gross Profit - Sales Last Month', 'Gross Profit - Sales Last 3 Months',
       'Gross Profit - Sales Last 6 Months',
       'Gross Profit - Sales Last 9 Mont

In [709]:
cols_to_rename = ['Gross Profit - Sales Last Month', 
       'Gross Profit - Sales Last 3 Months',
       'Gross Profit - Sales Last 6 Months',
       'Gross Profit - Sales Last 9 Months',
       'Gross Profit - Sales Last 12 Months',
       'Gross Profit - Sales Last 2 Years',
       'Gross Profit - Sales Last 3 Years',
       'Gross Profit - Sales Last 4 Years',
       'Gross Profit - Sales Last 5 Years',
       'Gross Profit - Sales Last 10 Years']

# Define a mapping of current column names to new column names
column_mapping = {col: col.replace('Sales', '') for col in cols_to_rename}

# Rename the columns using the .rename() method
parts_data.rename(columns=column_mapping, inplace=True)


In [710]:
def negative_sales(data_frame, column_names):
    """
    The reason there are negative sales numbers is to reflect inventory that was sold prior to being added correctly into the dealership ERP,
    this is common practice during busier months as volume dramatically increases and inventory can only be added to the system every so often.
    Additionally, inventory counts occur 1 - 2 times a year which is why many of the issues such as this persist across time

    Usage: provide dataframe and columns to switch and will set negative columns to their absolute value
    """

    data_frame = data_frame.copy()  # Make a copy to avoid the warning

    for col in column_names:
        if np.issubdtype(data_frame[col].dtype, np.number):  # Only apply to numeric columns
            data_frame.loc[data_frame[col] < 0, col] = abs(data_frame[col])

    return data_frame

# Usage:
column_names = ['Sales - Last Year', 
    'Sales - Jan', 'Sales - Feb', 'Sales - Mar',
    'Sales - Apr', 'Sales - May', 'Sales - Jun', 'Sales - Jul',
    'Sales - Aug', 'Sales - Sep', 'Sales - Oct', 'Sales - Nov', 'Sales - Dec',
    'Sales - This Year', 
    'Sales Last Month', 'Sales Last 3 Months', 'Sales Last 6 Months',
    'Sales Last 9 Months', 'Sales Last 12 Months', 'Sales Last 2 Years',
    'Sales Last 3 Years', 'Sales Last 4 Years', 'Sales Last 5 Years', 'Sales Last 10 Years']

parts_data = negative_sales(parts_data, column_names)

In [711]:
parts_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101559 entries, 0 to 103358
Data columns (total 66 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Part Number                     101559 non-null  object 
 1   Category                        101559 non-null  object 
 2   Description                     101559 non-null  object 
 3   Supplier Name                   101559 non-null  object 
 4   Margin %                        101559 non-null  float64
 5   Margin                          101559 non-null  float64
 6   Price                           101559 non-null  float64
 7   Quantity                        101559 non-null  float64
 8   Sales Last Month                101559 non-null  float64
 9   Sales Last 3 Months             101559 non-null  float64
 10  Sales Last 6 Months             101559 non-null  float64
 11  Sales Last 9 Months             101559 non-null  float64
 12  Sales Last 12 Mo

In [712]:
parts_data.describe()

,Margin %,Margin,Price,Quantity,Sales Last Month,Sales Last 3 Months,Sales Last 6 Months,Sales Last 9 Months,Sales Last 12 Months,Sales Last 2 Years,...,Gross Profit - Mar,Gross Profit - Apr,Gross Profit - May,Gross Profit - Jun,Gross Profit - Jul,Gross Profit - Aug,Gross Profit - Sep,Gross Profit - Oct,Gross Profit - Nov,Gross Profit - Dec
count,101559.000000,101559.000000,101559.000000,101559.000000,101559.000000,101559.000000,101559.000000,101559.000000,101559.000000,101559.000000,...,101559.000000,101559.000000,101559.000000,101559.000000,101559.000000,101559.000000,101559.0,101559.0,101559.0,101559.0
mean,40.267716,81.624103,222.430482,0.561414,0.053203,0.156439,0.261041,0.676832,0.775755,1.136482,...,4.892416,4.450103,5.469930,5.705775,4.843809,0.307080,0.0,0.0,0.0,0.0
std,312.001345,206.638692,563.648551,5.384358,1.318415,3.481379,5.594411,8.143394,9.539861,15.523013,...,68.768972,66.129359,73.893541,71.431664,77.560365,11.178608,0.0,0.0,0.0,0.0
min,-98600.000000,-3305.000000,0.010000,-12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-540.990000,-1256.990000,-1637.220000,-2015.040000,-579.990000,-389.990000,0.0,0.0,0.0,0.0
25%,33.080000,7.400000,19.990000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,38.600000,26.820000,70.990000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,49.070000,88.345000,235.990000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,100.000000,15299.000000,29060.990000,554.000000,216.000000,421.000000,695.000000,850.000000,1137.000000,2196.000000,...,5893.990000,4000.990000,4318.990000,4199.960000,10250.990000,1009.990000,0.0,0.0,0.0,0.0


In [713]:
mask = parts_data['Total Cost'] == 0.00

# Update 'Margin' column based on the condition
parts_data['Margin'] = np.where(mask, parts_data['Price'], parts_data['Margin'])

parts_data['Margin %'] = (parts_data['Margin'] / parts_data['Price']) * 100


In [714]:
#merged supplier column and added turnover
parts_data = pd.read_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/final_project/data/Project_Data/cleaned_parts_data.csv')

parts_data = parts_data.drop(columns=['Unnamed: 0'])

parts_data.drop_duplicates(subset=['Part Number'], inplace=True)

parts_data

,Part Number,Category,Description,Supplier Name,Margin %,Margin,Price,Quantity,Sales Last Month,Sales Last 3 Months,...,Gross Profit - Mar,Gross Profit - Apr,Gross Profit - May,Gross Profit - Jun,Gross Profit - Jul,Gross Profit - Aug,Gross Profit - Sep,Gross Profit - Oct,Gross Profit - Nov,Gross Profit - Dec
0,09148-04032,SHP,nut,**added from pricebook**,100.0,3.43,3.43,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0,0,0,0
1,09320-04501,SHP,cushion,**added from pricebook**,100.0,3.67,3.67,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0,0,0,0
2,09329-10026,SHP,cushion,**added from pricebook**,100.0,2.97,2.97,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0,0,0,0
3,10-08207-51,ACC,painted dsr jet black with,**added from pricebook**,100.0,2895.00,2895.00,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0,0,0,0
4,25-08031,ACC,fx/mx brake lever jjuan,**added from pricebook**,100.0,49.99,49.99,0.0,0.0,0.0,...,0.0,49.99,0.0,0.0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101554,Z5100,AL1,side illum. light set,ztechnik,100.0,479.00,479.00,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0,0,0,0
101555,Z5220,AOA,ztechnik® windscreen stabilizer kit,ztechnik,100.0,105.56,105.56,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0,0,0,0
101556,Z6192,AOA,accessory bar mount; 7/8inch,ztechnik,100.0,69.95,69.95,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0,0,0,0
101557,Z6201,AEL,gps/xm small access top plate,ztechnik,100.0,14.95,14.95,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0,0,0,0


In [715]:
#calculate inventory turnover
#formula: COGS / ((Goods at begging of time period - goods at end of time period) / 2)

def inventory_turn_total(cogs, end, start, ):
    """
        Gets the turnover for the inventory given a particular time period
        
        Usage: provide inventory amount at begining of period and end of period 
               and the cost of the goods sold over that period
    
        If the 'start' parameter is not provided, the function assumes that the beginning inventory is equal to the ending inventory.
    """
    average_inventory = (start + end) / 2
    
    turnover = cogs / average_inventory
    
    return turnover


In [716]:
# Calculate the required metrics in the parts_data DataFrame
total_cogs = round((parts_data['Cost Per Unit'] * parts_data['Sales - This Year']).sum(), 2)
total_end_inventory = round((parts_data['Price'] * parts_data['Quantity']).sum(), 2)
total_start_inventory = (total_end_inventory - parts_data['Quantity Received YTD'].sum()) + total_cogs
avg_total_inventory = round((total_start_inventory + total_end_inventory) / 2, 2)
total_turnover = total_cogs / avg_total_inventory if avg_total_inventory > 0 else 0

# Create a list of dictionaries for the tabular chart
data = [
    {
        "Metric": "Start Inventory",
        "Value": total_start_inventory
    },
    {
        "Metric": "End Inventory",
        "Value": total_end_inventory
    },
    {
        "Metric": "Average Inventory",
        "Value": avg_total_inventory
    },
    {
        "Metric": "COGS",
        "Value": total_cogs
    },
    {
        "Metric": "Turnover",
        "Value": total_turnover
    }
]

# Print the tabular chart
table = pd.DataFrame(data, columns=['Metric', 'Value'])
table

,Metric,Value
0,Start Inventory,3.081195e+06
1,End Inventory,2.271305e+06
2,Average Inventory,2.676250e+06
3,COGS,1.088888e+06
4,Turnover,4.068707e-01


In [717]:
def create_supplier_dataframe(parts_data, suppliers=None):
    """
    Create a data frame with the turnover, cogs, and average inventory for suppliers
    
    Usage: Takes 1 or more suppliers and returns a DataFrame with the corresponding values.
           The default value if no supplier is given is all the suppliers in the data set
    """
    if suppliers is None:
        suppliers = parts_data['Supplier Name'].unique()
    
    # Filter data based on suppliers
    suppliers_data = parts_data[parts_data['Supplier Name'].isin(suppliers)]
    
    # Calculate COGS, end inventory, start inventory, average inventory, and turnover
    suppliers_data['COGS'] = suppliers_data['Cost Per Unit'] * suppliers_data['Sales - This Year']
    suppliers_data['End Inventory'] = suppliers_data['Price'] * suppliers_data['Quantity']
    suppliers_data['Start Inventory'] = suppliers_data['End Inventory'] - suppliers_data['Quantity Received YTD'] + suppliers_data['COGS']
    suppliers_data['Average Inventory'] = (suppliers_data['Start Inventory'] + suppliers_data['End Inventory']) / 2
    suppliers_data['Turnover'] = np.where(suppliers_data['Average Inventory'] <= 0, 0, suppliers_data['COGS'] / suppliers_data['Average Inventory'])

    # Group by supplier and aggregate the data
    result_df = suppliers_data.groupby(['Supplier Name']).agg({
        'Average Inventory': 'mean',
        'COGS': 'sum',
        'Turnover': 'mean'
    }).reset_index()
    
    supplier_df = create_supplier_dataframe(parts_data)

In [737]:
def create_part_dataframe(parts_data, part_numbers=None):
    """
    Create a data frame with the turnover, cogs, and average inventory for individual part numbers
    
    Usage: Takes 1 or more part numbers and returns a DataFrame with the corresponding values.
           The default value if no part number is given is all the part numbers in the data set
    """
    if part_numbers is None:
        part_numbers = parts_data['Part Number'].unique()
    
    part_data = parts_data[parts_data['Part Number'].isin(part_numbers)]  # Filter the relevant data
    part_data['COGS'] = part_data['Cost Per Unit'] * part_data['Sales - This Year']
    part_data['End Inventory'] = part_data['Price'] * part_data['Quantity']
    part_data['Start Inventory'] = part_data['End Inventory'] - part_data['Quantity Received YTD'] + part_data['COGS']
    part_data['Average Inventory'] = (part_data['Start Inventory'] + part_data['End Inventory']) / 2
    part_data['Turnover'] = part_data.apply(
    lambda row: row['Average Inventory'] / row['Quantity'] if (row['COGS'] == 0) & (row['COGS'] == 0 and row['Quantity'] != 0)  else row['COGS'], axis=1)
    part_data.loc[part_data['End Inventory'] == 0, 'Turnover'] = 0
    part_data.loc[part_data['Average Inventory'] > 0, 'Turnover'] = part_data['COGS'] / part_data['Average Inventory']
    
    result_df = part_data.groupby(['Part Number', 'Description', 'Supplier Name', 'Quantity']).agg({
        'Start Inventory': 'sum',
        'Average Inventory': 'mean',
        'COGS': 'sum',
        'Turnover': 'mean'
    }).reset_index()
    
    return result_df

parts_turn = create_part_dataframe(parts_data)

In [738]:
#merge parts turnover with the parts data_set 
turnover_per_part = parts_turn[['Part Number', 'Turnover']]

complete_parts_data = pd.merge(parts_data, turnover_per_part, on='Part Number')

In [741]:
wrong_sales_data = complete_parts_data[(complete_parts_data['Turnover'] == 0) & (complete_parts_data['Months No Sale'] < 12) & (complete_parts_data['Quantity'] > 0)][['Part Number', 'Description', 
                    'Supplier Name', 'Price', 'Quantity', 'Months No Sale', 'Sales Last Month', 'Sales Last 3 Months', 
                    'Sales Last 6 Months', 'Sales Last 9 Months', 'Sales Last 12 Months', 'Sales - This Year', 'Turnover', 'Total Cost']]

#wrong_sales_data.to_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/final_project/data/Project_Data/wrong_sales_data.csv')

wrong_sales_data

,Part Number,Description,Supplier Name,Price,Quantity,Months No Sale,Sales Last Month,Sales Last 3 Months,Sales Last 6 Months,Sales Last 9 Months,Sales Last 12 Months,Sales - This Year,Turnover,Total Cost
1318,FM465-04RACE,"bmc,race,air,filter,675",bayside performance,139.99,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.39
1339,2035485,ps speed dial fit system,bell sports inc,4.99,1.0,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.96
1343,7072343,panovision shield clear,bell sports inc,109.95,1.0,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,69.97
1371,7092648,scout air blk lg,bell sports inc,209.99,1.0,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,114.70
1445,7121887,ps broozer xxl gloss blk,bell sports inc,299.95,1.0,0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,176.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89250,12-08196,xmx sds obd bypass kit,zero motorcycles,166.99,3.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,324.00
89282,24-08223,"reworked chin fairing right,",zero motorcycles,92.99,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.10
89285,24-08282-24,jet black tank panel,zero motorcycles,560.99,1.0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,364.41
89288,24-08299-51,charge tank cover,zero motorcycles,519.99,1.0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,337.91


In [733]:
cost_mask = complete_parts_data['Total Cost'] == 0

cols = ['Part Number', 'Description', 
                    'Supplier Name', 'Price', 'Quantity', 'Months No Sale', 'Sales Last Month', 'Sales Last 3 Months', 
                    'Sales Last 6 Months', 'Sales Last 9 Months', 'Sales Last 12 Months', 'Sales - This Year', 'Turnover', 'Total Cost']

complete_parts_data[cost_mask][cols]

,Part Number,Description,Supplier Name,Price,Quantity,Months No Sale,Sales Last Month,Sales Last 3 Months,Sales Last 6 Months,Sales Last 9 Months,Sales Last 12 Months,Sales - This Year,Turnover,Total Cost
0,09148-04032,nut,**added from pricebook**,3.43,0.0,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,09320-04501,cushion,**added from pricebook**,3.67,0.0,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,09329-10026,cushion,**added from pricebook**,2.97,0.0,28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10-08207-51,painted dsr jet black with,**added from pricebook**,2895.00,0.0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,25-08031,fx/mx brake lever jjuan,**added from pricebook**,49.99,0.0,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89401,Z5100,side illum. light set,ztechnik,479.00,0.0,203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89402,Z5220,ztechnik® windscreen stabilizer kit,ztechnik,105.56,0.0,47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89403,Z6192,accessory bar mount; 7/8inch,ztechnik,69.95,0.0,189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89404,Z6201,gps/xm small access top plate,ztechnik,14.95,0.0,188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [723]:
complete_parts_data.to_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/final_project/data/Project_Data/cleaned_parts_data.csv')